In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import *

In [0]:
init_load_flag = dbutils.widgets.get("init_load_flag")

####Data Reading From Source

In [0]:
df = spark.sql("select * from databricks_cata.silver.customers_silver")
#df.display()

###Removing Duplicates

In [0]:
df = df.dropDuplicates(subset=['customer_id'])
#df.limit(10).display()

##Dividing New vs Old Records 

In [0]:
if init_load_flag == 0:
   df_old = spark.sql('''select DimCustomerKey, customer_id, create_date, update_date 
                        from databricks_cata.gold.DimCustomers''')
else:
    df_old = spark.sql('''select 0 DimCustomerKey, 0 customer_id, 0 create_date, 0 update_date 
                        from databricks_cata.silver.customers_silver where 1=0''')

In [0]:
df_old.display()

####Renaming columns of df_old

In [0]:
df_old = df_old.withColumnRenamed("DimCustomerKey", "old_DimCustomerKey")\
    .withColumnRenamed("customer_id", "old_customer_id")\
    .withColumnRenamed("create_date", "old_create_date")\
    .withColumnRenamed("update_date", "old_update_date")
#df_old.limit(10).display()

###Applying Join With the Old Records

In [0]:
df_join = df.join(df_old, df['customer_id'] == df_old['old_customer_id'], 'left')

df_join.limit(10).display( )

###Separating new vs old records

In [0]:
df_new = df_join.filter(df_join['old_DImCustomerKey'].isNull())
#df_new.limit(10).display()

In [0]:
df_old = df_join.filter(df_join['old_DImCustomerKey'].isNotNull())

###Preparing df_old

In [0]:
# dropping all the columns which are nor required
df_old = df_old.drop("old_DimCustomerKey", "old_customer_id","old_update_date")

# Renaming old create date column to create_date

df_old = df.withColumnRenamed("old_create_date", "create_date")
df_old = df_old.withColumn("create_date", to_timestamp(col("create_date")))

#Recreating update column with current timestamp

df_old = df_old.withColumn("update_date", current_timestamp())

####Surrogate key - All the Values

In [0]:
df = df.withColumn("DimcCustomerKey", monotonically_increasing_id()+lit(1))
#df.limit(10).display()

In [0]:
df_old.display()